In [ ]:
# ########## auto login ############
# import login
# from login import *
# import pandas as pd
# from loguru import logger
# import pytz
# from time import sleep
# import datetime
# import json
# import pandas as pd
# logger.add("./logs/MNC_logs_{time}.log")

In [ ]:
# #######################
# # MANUAL LOGIN-1 of 1 #
# #######################

import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.ERROR)
import threading
kite = KiteConnect(api_key="w19o0chuo929jxkp")
import joblib
kite = joblib.load('kitefile.p')
print(kite.login_url())
mytoken = input('Enter Token Here : ')  
data = kite.generate_session(mytoken,api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
kite.set_access_token(data["access_token"])
import joblib
joblib.dump(kite,'kitefile.p')
import act
from act import *
api_k = "w19o0chuo929jxkp"  # api_key
api_s = "gsw8ps17ex7lf3cuji4prfnwb4vlyr4y"  # api_secret
my_ret_token = get_login(api_k, api_s)
kws = KiteTicker("w19o0chuo929jxkp", my_ret_token)
from loguru import logger
logger.add("./logs/SUPERTREND_logs_{time}.log")

In [ ]:
############## Parameters #######################
my_margin = 400
order_type = 'MIS'
exchange_type = kite.EXCHANGE_NSE
offset_quantity = 0
away_from_circuit = 0.50
########### Constants ##########
n50_token = 256265
n50_symbol = 'NIFTY 50'
days_ago=1

In [ ]:
###### Get Stocks Margin Map ############
import requests
url = 'https://api.kite.trade/margins/equity'
margin_data = requests.get(url, timeout=2.50)
margin_map = json.loads(margin_data.content.decode('utf-8'))
margin_map_df = pd.DataFrame(margin_map)

############ Get N/50 Stocks List ###########
n50df = pd.read_csv('n50.csv')
n50df = n50df.loc[n50df['tradable'] == 'y']

############ Merge Both Data ################
merged_df = pd.merge(n50df, margin_map_df, on="tradingsymbol")
merged_df

In [ ]:
############## Stock Margin Calucator for Bracket Order #####################
def calc_margin(co_upper,co_lower,price,stoploss=100, transaction_type = 'sell',quantity=1):
    '''calc_margin(co_upper,co_lower,stk_data,price,stoploss=10, transaction_type = 'sell',quantity=1)'''
    co_lower = co_lower/100
    co_upper = co_upper/100
    trigger = price - (co_upper * price)
    
    if stoploss < trigger:
        stoploss = trigger
    else:
        trigger = stoploss
    x = 0
    if transaction_type == 'buy':
        x = (price - trigger) * quantity
    else:
        x = (trigger - price) * quantity
        
    y = co_lower * price * quantity
    margin  = x if( x > y) else y
#     margin = x > y ? x : y
    margin = margin + (margin * 0.2)
    
#     y = round(y,4)

    return margin

############# Get 9:15 Data ##############################################
def get_mornign_data( inst_token, interval='5minute'):
    '''get_mornign_data( inst_token, interval='5minute')'''
    interval=interval
    inst_token = inst_token
    todaydt=datetime.date.today()
    hud_ago=todaydt-datetime.timedelta(days=1)
    to_date=datetime.date.isoformat(todaydt)

    from_date=datetime.date.isoformat(hud_ago)
    data = kite.historical_data(inst_token, from_date, to_date, interval, continuous=False)

    mydata = pd.DataFrame(data)
    today = datetime.date.today()

#     print('Today Date : '+str(today))
    yest_date =  mydata[-days_ago:][['date']].to_numpy()[0][0]
    last_close =  mydata[-days_ago:][['close']].to_numpy()[0][0]
    last_high =  mydata[-days_ago:][['high']].to_numpy()[0][0]
    last_low =  mydata[-days_ago:][['low']].to_numpy()[0][0]
#     print('yest_date : '+str(yest_date))
#     print('last close : '+ str(last_close))
#     print('last high : '+ str(last_high))
#     print('last low : '+ str(last_low))

    import pytz
    tz = pytz.timezone("Asia/Calcutta")
    b =tz.localize( datetime.datetime(today.year, today.month, today.day, 9, 15,0,0))
    # today.day
    timestamp = pd.Timestamp(b)

    mycandle = mydata.loc[mydata['date'] == timestamp]
    return {'open':mycandle['open'].to_numpy()[0],
            'high':mycandle['high'].to_numpy()[0],
            'low':mycandle['low'].to_numpy()[0],
            'close':mycandle['close'].to_numpy()[0],
           }

########### Decide Buy/Sell ###############
def decide_buy_sell(morning_candle):
    direction = ''
    if(morning_candle['open']>morning_candle['close']):
        direction = 'sell'
    if(morning_candle['open']<morning_candle['close']):
        direction = 'buy'
    return direction

In [ ]:
################ Wait Till 9:20 #####################
def act(x):
    return x+10

def wait_start(runTime, action):
    startTime = datetime.time(*(map(int, runTime.split(':'))))
    while startTime > datetime.datetime.today().time(): 
        sleep(1)
    return action

wait_start('9:20', lambda: act(100))

In [ ]:
################# Add Buy/Sell Direction to merged_df ############
direction_list = []
for i in merged_df.instrument_token.to_list():
    this_map = get_mornign_data(i)
    this_dir = decide_buy_sell(this_map)
    direction_list.append({'instrument_token':i,'direction':this_dir,
                           'morning_open':this_map['open'],
                           'morning_high':this_map['high'],
                           'morning_low':this_map['low'],
                           'morning_close':this_map['close']})
    print(i)

direction_list_df = pd.DataFrame(direction_list)
merged_df2 = pd.merge(merged_df, direction_list_df, on="instrument_token")
# merged_df2

In [ ]:
################ Get Considered DF Comparing to N50 Direction ########## 
nifty50_morning_candle = get_mornign_data(n50_token)
nifty_direction = decide_buy_sell(nifty50_morning_candle)
considered_df = merged_df2.loc[merged_df2['direction'] == nifty_direction]
# considered_df

In [ ]:
################### Decide Qty for Every Symbol #####################
total_margin_parts = len(considered_df.instrument_token)
per_symbol_margin = my_margin/total_margin_parts

qty_df_list = []

for inst in considered_df.instrument_token:
    this_co_upper = considered_df.loc[considered_df['instrument_token'] == inst]['co_upper'].to_numpy()[0]
    this_co_lower = considered_df.loc[considered_df['instrument_token'] == inst]['co_lower'].to_numpy()[0]
    this_price = 1.0
    this_sl = 1.0
    if(nifty_direction=='buy'):
        this_price = considered_df.loc[considered_df['instrument_token'] == inst]['morning_high'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] == inst]['morning_low'].to_numpy()[0]
    else:
        this_price = considered_df.loc[considered_df['instrument_token'] == inst]['morning_low'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] == inst]['morning_high'].to_numpy()[0]
    this_marg = calc_margin(this_co_upper,this_co_lower,this_price,stoploss=this_sl,
                            transaction_type = nifty_direction) 
    this_qty = int(per_symbol_margin/this_marg)
    qty_df_list.append({'instrument_token':inst,'co_bo_margin':this_marg,
                        'trade_qty':this_qty,'my_qty':0,'ltp':0})

qty_df = pd.DataFrame(qty_df_list)
print('Per Symbol Margin : '+str(per_symbol_margin))

qty_df = pd.merge(considered_df,qty_df, on="instrument_token")
qty_df

In [ ]:
############# Get My Qty ########################################
myquantity = 0
def getquant():
    global myquantity
    global order_type
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        for j in qty_df.instrument_token.to_list():
            if(allpos[i]['instrument_token']==j and  allpos[i]['product']== 'CO'):
                myquantity = allpos[i]['quantity']
                qty_df.at[qty_df.index[qty_df['instrument_token'] == j].tolist()[0],'my_qty']= myquantity
                this_smb = qty_df.loc[qty_df['instrument_token'] == j]['tradingsymbol'].to_numpy()[0]
                print('My Quantity : '+ this_smb + ' : ' + str(allpos[i]['quantity']))
getquant()
################# Place Order #####################################
stopbuy = False
stopsell = False
placing_order = False
def placeneworder(inst_token,quantdiff,trigger_price):
    '''placeneworder(inst_token,quantdiff,trigger_price)'''
    global stopbuy,stopsell,placing_order
    my_symbol_ip = qty_df.loc[qty_df['instrument_token'] == inst_token]['tradingsymbol'].to_numpy()[0]
        
    if(quantdiff>0 and stopbuy == False):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                        exchange=kite.EXCHANGE_NSE, 
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        trigger_price = trigger_price,
                        quantity=abs(quantdiff),
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_CO,
                        product=kite.PRODUCT_CO,)
            getquant()
            stopsell = False
        except Exception as e:
            stopbuy = False
            print(e)
            getquant()
        placing_order = False
    if(quantdiff<0 and stopsell == False):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                    exchange=kite.EXCHANGE_NSE, 
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    trigger_price = trigger_price,
                    quantity=abs(quantdiff),
                    order_type=kite.ORDER_TYPE_MARKET,
                    variety = kite.VARIETY_CO,
                    product=kite.PRODUCT_CO,)
            getquant()
            stopbuy = False
        except Exception as e:
            stopsell = True
            print(e)
            getquant()
        placing_order = False
############### CHK QTY #####################################
def ckqnt (tkn,orderquant,trigger_price):
    '''ckqnt (tkn,orderquant,trigger_price)'''
    global placing_order
    myquantity=qty_df.loc[qty_df['instrument_token'] == tkn]['my_qty'].to_numpy()[0]
    quantdiff = orderquant - myquantity
    if(placing_order==False and quantdiff!=0):
        placeneworder(inst_token=tkn,quantdiff=quantdiff,trigger_price=trigger_price)
    return myquantity

In [ ]:
############# Start Trading ###############################
import logging
import threading
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)

def on_ticks(ws, ticks):
    print('==========================')
    this_min = datetime.datetime.now().minute
    this_hour = datetime.datetime.now().hour
    
    for one_stk_data in ticks:
        print('Symbol : '+str(qty_df.loc[qty_df['instrument_token'] == one_stk_data['instrument_token']]
                              ['tradingsymbol'].to_numpy()[0]))
        print('LTP : '+str(one_stk_data['last_price']))
        this_ltp = one_stk_data['last_price']
        this_morning_low = qty_df.loc[qty_df['instrument_token'] == one_stk_data['instrument_token']]['morning_low'].to_numpy()[0]
        this_morning_high = qty_df.loc[qty_df['instrument_token'] == one_stk_data['instrument_token']]['morning_high'].to_numpy()[0]
        this_stk_qty = qty_df.loc[qty_df['instrument_token'] == one_stk_data['instrument_token']]['trade_qty'].to_numpy()[0]
        
        qty_df.at[qty_df.index[qty_df['instrument_token'] ==  one_stk_data['instrument_token']]
                  .tolist()[0],'ltp']= this_ltp
        
        if(nifty_direction=='sell'):
            print('Morning Low : '+str(this_morning_low))
        else:
            print('Morning High : '+str(this_morning_high))
        print('*****************************')
        
        if(this_hour<=11 and this_min<=1 and this_stk_qty!=0):
            if(nifty_direction=='sell' and this_ltp<this_morning_low ):
                ckqnt (one_stk_data['instrument_token'],-this_stk_qty,this_morning_high)
            if(nifty_direction=='buy' and this_ltp>this_morning_high):
                ckqnt (one_stk_data['instrument_token'],this_stk_qty,this_morning_low)

    qty_df.to_excel('./live_data.xlsx')
    print('==========================')
    print()

def on_connect(ws, response):
    ws.subscribe(qty_df.instrument_token.to_list())
    ws.set_mode(ws.MODE_FULL,qty_df.instrument_token.to_list())

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    print("order update: ", data)
    
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [ ]:
# kite.place_order(
#     tradingsymbol = 'ONGC',
#     transaction_type =kite.TRANSACTION_TYPE_BUY,
#     trigger_price = 70.00,
#     quantity = 1,
#     order_type=kite.ORDER_TYPE_MARKET,
#     variety=kite.VARIETY_CO,
#     exchange=kite.EXCHANGE_NSE,
#     product=kite.PRODUCT_CO)

In [ ]:
# inst = kite.instruments()
# leninst = len(inst)
# n50list = []
# for i in range(leninst):
#     for j in range(len(nifty50)):
#         if(inst[i]['tradingsymbol']==nifty50[j] and inst[i]['exchange']=='NSE'):
#             n50list.append(inst[i])
# n50list
# mydf = pd.DataFrame(n50list)  
# mydf.to_csv('n50.csv')  